In [1]:
import os
import string
import datetime
import regex as re
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

import plotly.plotly as py
import plotly.graph_objs as go

plt.style.use('fivethirtyeight')
#%matplotlib inline

In [2]:
from nltk.corpus import stopwords

In [3]:
ira_tweets = pd.read_csv('../data/ira_tweets_csv_hashed.csv')

/Users/gcdunn/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning:

Columns (30) have mixed types. Specify dtype option on import or set low_memory=False.



In [4]:
ira_tweets.head()

,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,user_profile_url,follower_count,following_count,account_creation_date,...,latitude,longitude,quote_count,reply_count,like_count,retweet_count,hashtags,urls,user_mentions,poll_choices
0,877919995476496385,249064136b1c5cb00a705316ab73dd9b53785748ab757f...,249064136b1c5cb00a705316ab73dd9b53785748ab757f...,249064136b1c5cb00a705316ab73dd9b53785748ab757f...,"Москва, Россия",Я примерный семьянин!,NaN,132,120,2013-12-07,...,absent,absent,0.0,0.0,0.0,0.0,[],[http://ru-open.livejournal.com/374284.html],[2572896396],NaN
1,492388766930444288,0974d5dbee4ca9bd6c3b46d62a5cbdbd5c0d86e196b624...,0974d5dbee4ca9bd6c3b46d62a5cbdbd5c0d86e196b624...,0974d5dbee4ca9bd6c3b46d62a5cbdbd5c0d86e196b624...,Россия,Телефонист .Изучение истории Игра в любитель...,NaN,74,8,2014-03-15,...,absent,absent,0.0,0.0,0.0,0.0,NaN,[http://pyypilg33.livejournal.com/11069.html],NaN,NaN
2,719455077589721089,bda40f262856eee77c48a332e5eb23bc4f1943d600867d...,bda40f262856eee77c48a332e5eb23bc4f1943d600867d...,bda40f262856eee77c48a332e5eb23bc4f1943d600867d...,Рязань,волны так и плещут фиолетовой волной,NaN,165,454,2014-04-29,...,absent,absent,0.0,0.0,0.0,0.0,[],[https://www.youtube.com/watch?v=9GvpImWxTJc],[40807205],NaN
3,536179342423105537,bda40f262856eee77c48a332e5eb23bc4f1943d600867d...,bda40f262856eee77c48a332e5eb23bc4f1943d600867d...,bda40f262856eee77c48a332e5eb23bc4f1943d600867d...,Рязань,волны так и плещут фиолетовой волной,NaN,165,454,2014-04-29,...,absent,absent,0.0,0.0,0.0,0.0,[STOPNazi],NaN,NaN,NaN
4,841410788409630720,a53ed619f1dea6015c7c878bf744b0eefe8f7272dccf34...,a53ed619f1dea6015c7c878bf744b0eefe8f7272dccf34...,a53ed619f1dea6015c7c878bf744b0eefe8f7272dccf34...,NaN,Отвечаю на любой #ВопросПрезиденту,http://t.co/3CVqbMQFbf,4430,4413,2012-02-25,...,absent,absent,0.0,0.0,3.0,4.0,[],[https://goo.gl/fBp94X],NaN,NaN


In [5]:
ira_tweets.columns

Index(['tweetid', 'userid', 'user_display_name', 'user_screen_name',
       'user_reported_location', 'user_profile_description',
       'user_profile_url', 'follower_count', 'following_count',
       'account_creation_date', 'account_language', 'tweet_language',
       'tweet_text', 'tweet_time', 'tweet_client_name', 'in_reply_to_tweetid',
       'in_reply_to_userid', 'quoted_tweet_tweetid', 'is_retweet',
       'retweet_userid', 'retweet_tweetid', 'latitude', 'longitude',
       'quote_count', 'reply_count', 'like_count', 'retweet_count', 'hashtags',
       'urls', 'user_mentions', 'poll_choices'],
      dtype='object')

In [6]:
ira_tweets.user_display_name

0          249064136b1c5cb00a705316ab73dd9b53785748ab757f...
1          0974d5dbee4ca9bd6c3b46d62a5cbdbd5c0d86e196b624...
2          bda40f262856eee77c48a332e5eb23bc4f1943d600867d...
3          bda40f262856eee77c48a332e5eb23bc4f1943d600867d...
4          a53ed619f1dea6015c7c878bf744b0eefe8f7272dccf34...
5          a53ed619f1dea6015c7c878bf744b0eefe8f7272dccf34...
6          95b3aba6b9140f5dda993148de174ff57d62f4a6e68e88...
7          5744c546bdf9e81ea0aad223c9db4b702ccba7c81d4c11...
8          2b0d7525bed1df5119b7956f9be4888b45686172d68006...
9          b88fd4fc4b169f0a98eb38d3f5ef72a1eb3f6861cb3e81...
10         be1e8e099e93fe0964c1184173f038c7413def3baf38ef...
11         be1e8e099e93fe0964c1184173f038c7413def3baf38ef...
12         be1e8e099e93fe0964c1184173f038c7413def3baf38ef...
13         f9dded769bb2275fc6531c1b4d10cf05272c48806fbd73...
14         f9dded769bb2275fc6531c1b4d10cf05272c48806fbd73...
15         f9dded769bb2275fc6531c1b4d10cf05272c48806fbd73...
16         f9dded769bb22

In [7]:
ira_tweets.account_language.value_counts()

en       4606393
ru       3953675
de        102657
zh-cn      29413
id         22966
en-gb      21494
es         13979
it         13105
ar          3452
fr          1409
uk            90
Name: account_language, dtype: int64

Only examine tweets in English

In [133]:
eng_tweets = ira_tweets[(ira_tweets.account_language == 'en') | (ira_tweets.account_language == 'en-gb')]

In [134]:
eng_tweets.account_language.value_counts()

en       4606393
en-gb      21494
Name: account_language, dtype: int64

Only examine accounts that are not hashed

In [136]:
eng_tweets = eng_tweets[eng_tweets.user_screen_name.str.len() <= 15]

In [137]:
eng_tweets.user_screen_name.head()

29     NovostiVolga
30     NovostiVolga
31     NovostiVolga
105     Jenn_Abrams
106     Jenn_Abrams
Name: user_screen_name, dtype: object

In [138]:
eng_tweets.tweet_text.head()

29     Сергей Горняков провел прием граждан в Урюпинс...
30        #волгоград #сталинград https://t.co/RCoCV12FP0
31           Красота нашего края https://t.co/4zMKgdCpYp
105    #IfAnimalsMadeLaws all parrots would be equal ...
106    Mexico City metro now has a penis seat to comb...
Name: tweet_text, dtype: object

Some of the tweets reported as English language text clearly are not.  We need a better way to sort out tweets in English.  Tweets written in English likely will not contain Cyrillic letters.  They will, however, likely contain English-language stopwords.

In [139]:
# tweets in English will not contain Cyrillic characters
def has_cyrillic(text):
    return bool(re.search('[\u0400-\u04FF]', text))

# tweets in English will contain stop words
stop = set(stopwords.words('english'))
def has_stopwords(text):
    if any(word in stop for word in text): 
        return 1
    else: return 0

In [140]:
eng_tweets = eng_tweets[eng_tweets.tweet_text.astype(str).apply(has_cyrillic) == 0]
eng_tweets = eng_tweets[eng_tweets.tweet_text.astype(str).apply(has_stopwords) == 1]

In [141]:
eng_tweets.tweet_text.head()

105    #IfAnimalsMadeLaws all parrots would be equal ...
106    Mexico City metro now has a penis seat to comb...
107    And will replace it with "Magister" Because yo...
108    Muslims can't even coexist with Muslims Why do...
109    @CapLarryDart @Co_Passenger won't stricter gun...
Name: tweet_text, dtype: object

In [142]:
eng_tweets.shape

(1179757, 31)

In [143]:
eng_tweets.to_csv('../data/all_eng_tweets.csv')

In [144]:
eng_tweets['tweet_time'] = pd.to_datetime(eng_tweets['tweet_time'])

Remove website links from tweet text

In [145]:
eng_tweets['tweet_text'] = eng_tweets.tweet_text.str.replace('https?://[A-Za-z0-9./]+','')

Remove hashtags from tweet text.  They are already stored in the column 'hashtags'.

In [146]:
eng_tweets['tweet_text'] = eng_tweets.tweet_text.str.replace('\#[A-Za-z0-9./]+','')

In [147]:
eng_tweets.tweet_text.head(20)

105      all parrots would be equal from the beginning  
106    Mexico City metro now has a penis seat to comb...
107    And will replace it with "Magister" Because yo...
108    Muslims can't even coexist with Muslims Why do...
109    @CapLarryDart @Co_Passenger won't stricter gun...
110            And who will determine what exactly is ? 
111    They finally admit they just need everything f...
112                                       @BBCWorld yeap
113    You know that fat girls who believe in  usuall...
114                             If you know what I mean 
115                          Have a nice Tuesday, guys! 
119                My attitude to feminism in one pic   
121                         Britain is the new Russia 😅 
122    I've noticed that stupid women usually call th...
155    Judge Mike Erwin still keeps his post after ma...
156    Getting richer will provide our communities mo...
176    Horse pulling carriage in  appears to collapse...
181    John Carroll University 

Remove @ mentions and RT @ and store them in a separate column

In [148]:
eng_tweets['mentions'] = eng_tweets.tweet_text.str.findall('(?<!RT\s)(@[A-Za-z0-9-_]+)')
eng_tweets['retweets'] = eng_tweets.tweet_text.str.findall('(?<=RT\s)(@[A-Za-z0-9-_]+)')

Save a dataframe of just hashtags, just mentions, and just retweeted users

In [149]:
#hashtags = pd.Series(eng_tweets.hashtags.dropna())
#hashtags = hashtags[hashtags.astype(str) != '[]']
#hashtags = hashtags.str.replace('[\[\]]', '')


#flattened_hashtags = pd.DataFrame(
#    [str(hashtag) for hashtags_list in hashtags.values
#    for hashtag in hashtags_list.split(',')],
#    columns=['hashtag'])

#flattened_hashtags.head()
#flattened_hashtags.to_csv('../data/all_eng_hashtags.csv')

In [150]:
#retweets = eng_tweets.retweets[eng_tweets.retweets.astype(str) != '[]']
#retweets = retweets.str.replace('[\[\]]', '')


#flattened_retweets = pd.DataFrame(
#    [str(retweet) for retweet_list in retweets.values
#    for retweet in retweet_list.split(',')],
#    columns=['retweets'])

#flattened_retweets.to_csv('../data/all_eng_retweets.csv')

In [151]:
#mentions = eng_tweets.mentions[eng_tweets.mentions.astype(str) != '[]']
#mentions = mentions.str.replace('[\[\]]', '')


#flattened_mentions = pd.DataFrame(
#    [str(mention) for mention_list in mentions.values
#    for mention in mention_list.str.split(',')],
#    columns=['mentions'])

#flattened_mentions.to_csv('../data/all_eng_mentions.csv')

### Make a document of tweets for each handle

In [152]:
#Remove users that only tweeted once?
#eng_tweets = eng_tweets[eng_tweets.groupby('user_screen_name')['user_screen_name'].transform('size')>1]

Remove retweet preface (RT :)

In [153]:
eng_tweets['tweet_text'] = eng_tweets.tweet_text.str.replace('RT', '')

Remove punctuation

In [154]:
eng_tweets.tweet_text = eng_tweets.tweet_text.apply(lambda x: re.sub('[@.,:;?!-*<>\'\"]', '', str(x)))

In [155]:
eng_tweets.head()

,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,user_profile_url,follower_count,following_count,account_creation_date,...,quote_count,reply_count,like_count,retweet_count,hashtags,urls,user_mentions,poll_choices,mentions,retweets
105,626740121648259072,2882331822,Jenna Abrams,Jenn_Abrams,USA,"Calm down, I'm not pro-Trump. I am pro-common ...",https://t.co/ZXoG0JPLdf,79152,22607,2014-10-29,...,0.0,0.0,2.0,1.0,"[IfAnimalsMadeLaws, EqualityForAll]",[],NaN,NaN,[],[]
106,851098834176487428,2882331822,Jenna Abrams,Jenn_Abrams,USA,"Calm down, I'm not pro-Trump. I am pro-common ...",https://t.co/ZXoG0JPLdf,79152,22607,2014-10-29,...,5.0,13.0,12.0,13.0,[],[],NaN,NaN,[],[]
107,851825187267989505,2882331822,Jenna Abrams,Jenn_Abrams,USA,"Calm down, I'm not pro-Trump. I am pro-common ...",https://t.co/ZXoG0JPLdf,79152,22607,2014-10-29,...,1.0,5.0,14.0,7.0,[],[],NaN,NaN,[],[]
108,681897201761697792,2882331822,Jenna Abrams,Jenn_Abrams,USA,"Calm down, I'm not pro-Trump. I am pro-common ...",https://t.co/ZXoG0JPLdf,79152,22607,2014-10-29,...,0.0,100.0,550.0,610.0,[],[],NaN,NaN,[],[]
109,662532293613875201,2882331822,Jenna Abrams,Jenn_Abrams,USA,"Calm down, I'm not pro-Trump. I am pro-common ...",https://t.co/ZXoG0JPLdf,79152,22607,2014-10-29,...,0.0,0.0,1.0,0.0,[],[],"[3296320187, 164332083]",NaN,"[@CapLarryDart, @Co_Passenger]",[]


In [156]:
user_rt = eng_tweets[eng_tweets.is_retweet==1].groupby('user_display_name').agg({'user_screen_name': lambda x: x.iloc[0],'tweet_text': lambda x: ''.join(x)})
user_original = eng_tweets[eng_tweets.is_retweet==0].groupby('user_display_name').agg({'user_screen_name': lambda x: x.iloc[0],'tweet_text': lambda x: ''.join(x)})

In [157]:
user_rt.head()

,user_screen_name,tweet_text
user_display_name,,
1-800-WOKE-AF,gloed_up,KatEdmiston gloed_up its not either/or Love e...
10 GOP,10_gop,10_gop Steve Bannon on Hillary Clinton Shes n...
Amy Green,AmandaVGreen,DynaMoChen Im a bitter-clinger - WorldOfHash...
Andy Sparks,AndyHashtagger,eddywaraich91 I could stop the bomb trgore ...
Atlanta Online,Atlanta_Online,ajc 2 people killed in Gwinnett crash followi...


In [158]:
eng_tweets.is_retweet.value_counts()

False    1031867
True      147890
Name: is_retweet, dtype: int64

In [159]:
user_original.to_csv('../data/user_original.csv')
user_rt.to_csv('../data/user_rt.csv')